<a href="https://colab.research.google.com/github/AsteriskAzurain/illuminera/blob/main/CodeBak_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copy from [Baidu AI Studio](https://aistudio.baidu.com/aistudio/projectdetail/408093)



### [Dataset Introduction](https://keras.io/datasets/)

Dataset of 11,228 newswires from Reuters, labeled over 46 topics. As with the IMDB dataset, each wire is encoded as a sequence of word indexes (same conventions).

**Usage:**

```python
from keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)
```

The specifications are the same as that of the IMDB dataset, with the addition of:

- **test_split**: float. Fraction of the dataset to be used as test data.

This dataset also makes available the word index used for encoding the sequences:

```
word_index = reuters.get_word_index(path="reuters_word_index.json")
```

- **Returns:** A dictionary where key are words (str) and values are indexes (integer). eg. `word_index["giraffe"]` might return `1234`.
- **Arguments:**
  - **path**: if you do not have the index file locally (at `'~/.keras/datasets/' + path`), it will be downloaded to this location.

  - **Returns:**
  - 2 tuples:
    - **x_train, x_test**: list of sequences, which are lists of indexes (integers). If the num_words argument was specific, the maximum possible index value is num_words-1. If the maxlen argument was specified, the largest possible sequence length is maxlen.
    - **y_train, y_test**: list of integer labels (1 or 0).
- **Arguments:**
  - **path**: if you do not have the data locally (at `'~/.keras/datasets/' + path`), it will be downloaded to this location.
  - **num_words**: integer or None. Top most frequent words to consider. Any less frequent word will appear as `oov_char` value in the sequence data.
  - **skip_top**: integer. Top most frequent words to ignore (they will appear as `oov_char` value in the sequence data).
  - **maxlen**: int. Maximum sequence length. Any longer sequence will be truncated.
  - **seed**: int. Seed for reproducible data shuffling.
  - **start_char**: int. The start of a sequence will be marked with this character. Set to 1 because 0 is usually the padding character.
  - **oov_char**: int. words that were cut out because of the `num_words` or `skip_top` limit will be replaced with this character.
  - **index_from**: int. Index actual words with this index and higher.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Tensorflow 2.0 initialization

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

import datetime, os


### Tensorboard

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir)



In [ ]:
%tensorboard --logdir logs


### Data Preparation

In [ ]:
num_epochs = 200
batch_size = 128
learning_rate = 0.001

num_classes = 46

In [ ]:
total_words = 40000
max_news_words = 400
embedding_len = 200

In [ ]:
(x_train, y_train),(x_test, y_test) = keras.datasets.reuters.load_data(num_words = total_words)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
## pad sequence to the same length
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen = max_news_words)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen = max_news_words)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
len(x_test[51])

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
res = next(iter(ds_train))
res[0].shape, res[1].shape

In [ ]:
def preprocess(x,y):
  y = tf.one_hot(y, depth=num_classes)
  return x, y

In [ ]:
# shuffle and batch dataset and drop the last batch shorter than batch_size
ds_train = ds_train.shuffle(1000).map(preprocess).batch(batch_size, drop_remainder = True)
ds_test = ds_test.shuffle(1000).map(preprocess).batch(batch_size, drop_remainder = True)

### RNN Models

#### SimpleRNN

In [ ]:
class RNN(keras.Model):
  def __init__(self, num_units, num_classes):
    super().__init__()
    # embedding [b, 200] -> [b, 200, 100]
    self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_news_words)

    self.RNN1 = layers.SimpleRNN(num_units, dropout = 0.5, return_sequences = True)
    self.RNN2 = layers.SimpleRNN(num_units, dropout = 0.5, return_sequences = False)

    self.fc = layers.Dense(num_classes)

  def call(self, inputs, training = None):
    
    outputs = self.embedding(inputs)
    
    outputs = self.RNN1(outputs, training = training)
    outputs = self.RNN2(outputs, training = training)
    outputs = self.fc(outputs)

    return outputs


#### LSTM

In [ ]:
class LSTM(keras.Model):
  def __init__(self, num_units, num_classes):
    super().__init__()
    # embedding [b, 200] -> [b, 200, 100]
    self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_news_words)

    self.RNN1 = layers.LSTM(num_units, dropout = 0.5, return_sequences = True)
    self.RNN2 = layers.LSTM(num_units, dropout = 0.5, return_sequences = False)

    self.fc = layers.Dense(num_classes)

  def call(self, inputs, training = None):
    
    outputs = self.embedding(inputs)
    
    outputs = self.RNN1(outputs, training = training)
    outputs = self.RNN2(outputs, training = training)
    outputs = self.fc(outputs)

    return outputs

#### GRU

In [ ]:
class GRU(keras.Model):
  def __init__(self, num_units, num_classes):
    super().__init__()
    # embedding [b, 200] -> [b, 200, 100]
    self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_news_words)

    self.RNN1 = layers.GRU(num_units, dropout = 0.5, return_sequences = True)
    self.RNN2 = layers.GRU(num_units, dropout = 0.5, return_sequences = True)
    self.RNN3 = layers.GRU(num_units, dropout = 0.5, return_sequences = False)

    self.fc = layers.Dense(num_classes)

  def call(self, inputs, training = None):
    
    outputs = self.embedding(inputs)
    
    outputs = self.RNN1(outputs, training = training)
    outputs = self.RNN2(outputs, training = training)
    outputs = self.RNN3(outputs, training = training)
    outputs = self.fc(outputs)

    return outputs

#### Keras Quick Training

In [ ]:
model = GRU(64,46)
model.build(input_shape=(None, max_news_words))
model.summary()

model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss = tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'])
history = model.fit(ds_train, epochs=100, validation_data=ds_test)

#### TF2.0 Model

In [ ]:
model = GRU(64,46)
model.build(input_shape=(None, max_news_words))
model.summary()

#### Initialize the optimizer


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate= learning_rate)

#### Train Evaluate and log the model in tensorboard


In [ ]:
categorical_accuracy_train = keras.metrics.CategoricalAccuracy()
categorical_accuracy_test = keras.metrics.CategoricalAccuracy()


In [ ]:
for epoch in range(num_epochs):
  for step, (x, y) in enumerate(ds_train):
    with tf.GradientTape() as tape:
      logits = model(x, training = True)
      loss = tf.keras.losses.categorical_crossentropy(y_true=y, y_pred=logits, from_logits = True)
      loss = tf.reduce_mean(loss)  
    
    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars= zip(grad, model.trainable_variables))

    [x_train, y_train] = next(iter(ds_train))
    train_logits = model(x_train, training = False)
    categorical_accuracy_train.update_state(y_true = y_train, y_pred=train_logits)
    train_accuracy = categorical_accuracy_train.result().numpy()
    
    [x_test, y_test] = next(iter(ds_test))
    logits = model(x_test, training = False)
    categorical_accuracy_test.update_state(y_true = y_test, y_pred=logits)
    accuracy = categorical_accuracy_test.result().numpy()

    if step%20 == 0:
      print("epoch: {}, step: {}, loss: {}, train_accuracy: {} test_accuracy: {}".format(epoch, step, loss.numpy(),train_accuracy,accuracy))

      # with summary_writer.as_default():
      #   tf.summary.scalar("loss epoch: "+str(epoch), loss.numpy(), step = step)
      #   tf.summary.scalar("test_acc epoch: "+str(epoch), accuracy, step = step)
  with summary_writer.as_default():
    tf.summary.scalar("epoch_loss", loss.numpy(), step=epoch)
    tf.summary.scalar("epoch_train_acc",train_accuracy, step=epoch)
    tf.summary.scalar("epoch_test_acc", accuracy, step=epoch)

In [ ]:
import pandas as pd
import os
import numpy as np
! pip install xlrd

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 112kB 10.4MB/s eta 0:00:01


In [ ]:
myfile='data/信息.xlsx'
pwd=os.getcwd()
pwd

In [ ]:
data=pd.read_excel(io=myfile,header=0)

In [ ]:
data[:5]

,StaffName_Chinese,StaffName_English,E_Name,AD_Account,Code
0,陈彦慧,Lynn Chen,Lynn Chen,Lynn.Chen,Lynn.Chen
1,周娉,Evan Zhou,Evan Zhou,Evan.Zhou,Evan.Zhou
2,蔡奕,Carol Cai,Carol Cai,Carol.Cai,Carol.Cai
3,董巍,David Dong,David Dong,David.Dong,David.Dong
4,周波,Paul Zhou,Paul Zhou,Paul.Zhou,Paul.Zhou


In [ ]:
data.columns.values
#data['Unnamed: 5'][2]

array(['StaffName_Chinese', 'StaffName_English', 'E_Name', 'AD_Account',
       'Code'], dtype=object)

In [ ]:
for i in range(len(data)):
    userid=data['id'][i]
    oldID=data['PositionID'][i]
    value=data['Unnamed: 5'][i]
    newID=0 if str(value)=='nan' else int(value)
    # print(userid,oldID,newID)
    sql_str="update t_User_UserInfo set PositionID={1} where ID={0}".format(userid,oldID)
    print(sql_str)

In [ ]:
idarr=[]
for i in range(len(data)):
    userid=str(data['id'][i])
    idarr.append(userid)
idstr=','.join(idarr)
print(idstr)


In [ ]:
import pymssql

In [ ]:
# server    数据库服务器名称或IP
# user      用户名
# password  密码
# database  数据库名称
server='192.168.1.245'
user='sa'
password='Illuminera2011'
database='IllumineraERP'
conn = pymssql.connect(server, user, password, database)

In [ ]:
cursor = conn.cursor()

In [ ]:
# myfile='data/mentee.xlsx'
myfile='data/pydata.xlsx'
staffinfo_file='data/staffinfo.xlsx'

In [ ]:
staff_dict={}
staff_data=pd.read_excel(io=staffinfo_file,header=0)
print(staff_data.columns.values)

for i in range(len(staff_data)):
    pk=int(staff_data['HR_StaffInfoID'][i])
    name=str(staff_data['StaffName_English'][i]).strip()
    staff_dict.update({name:pk})
len(staff_dict)
print(staff_dict['Aaron Guo'])

['HR_StaffInfoID' 'StaffName_English']
72


In [ ]:
data=pd.read_excel(io=myfile,header=0)
data[:5]

NameError: name 'pd' is not defined

In [ ]:
data.columns.values

array(['mentorname', 'HR_StaffInfoID', 'menteename', 'MenteeID',
       'StartDate', 'IsDeleted', 'CreateDate', 'CreateUserID',
       'CurrentStatus'], dtype=object)

In [ ]:
sql5="select m.HR_MenteeID from t_HR_Mentee m left join t_HR_StaffInfo s1 on m.HR_StaffInfoID=s1.HR_StaffInfoID left join t_HR_StaffInfo s2 on m.MenteeID=s2.HR_StaffInfoID"
sql4=sql5+" where s1.StaffName_English='{}' and s2.StaffName_English='{}'"
menteelist=[]
flist2=[]
for i in range(len(data)):
    m1=data['Mentor Name'][i]
    m2=data['MenteeName'][i]
    menteelist.append(m2)
    s=sql4.format(m1,m2)
    flist2.append(s)
for ss in flist2:
    print(ss)

sql temp
```SQL
select 
	m.HR_MenteeID ,
	m.HR_StaffInfoID as 'mentor ID', 
	s1.StaffName_English as 'Mentor Name',
	s1.StaffName_Chinese as 'Mentor C_Name', 
	m.MenteeID as 'Mentee ID', 
	s2.StaffName_English as 'Mentee Name',
	s2.StaffName_Chinese as 'Mentee C_Name',
	m.StartDate, m.EndDate, m.CurrentStatus
from t_HR_Mentee m
	left join t_HR_StaffInfo s1 on m.HR_StaffInfoID=s1.HR_StaffInfoID
	left join t_HR_StaffInfo s2 on m.MenteeID=s2.HR_StaffInfoID
where m.IsDeleted=1 and m.CurrentStatus=1
ORDER BY
	(
		CASE s2.StaffName_English
		WHEN '高' THEN 3
		WHEN '中' THEN 2
		ELSE 123
		END
	) ASC
```

In [ ]:
sql1="""
select 
	m.HR_MenteeID ,
	s1.StaffName_English as 'Mentor Name',
	s2.StaffName_English as 'Mentee Name',
	m.StartDate, m.EndDate
from t_HR_Mentee m
	left join t_HR_StaffInfo s1 on m.HR_StaffInfoID=s1.HR_StaffInfoID
	left join t_HR_StaffInfo s2 on m.MenteeID=s2.HR_StaffInfoID 
"""
sql2="where s2.StaffName_English = '{}'"
sql3="WHEN '{}' THEN {}"
flist=[]
i=1
for m in menteelist:
    # newstr=sql2.format(m)
    newstr=sql3.format(m,i)
    i+=1
    flist.append(newstr)
# len(flist)
# print(sql1+' and '.join(flist))
# print(sql1)
for sql in flist:
    print(sql)

WHEN 'Karen Duan' THEN 1
WHEN 'Stella Yang' THEN 2
WHEN 'Arlene Xu' THEN 3
WHEN 'M LU' THEN 4
WHEN 'Yulin Fei' THEN 5
WHEN 'Ariel Chen' THEN 6
WHEN 'Rose Feng' THEN 7
WHEN 'Collin Zeng' THEN 8
WHEN 'Chuting Chang' THEN 9
WHEN 'Yang Yang' THEN 10
WHEN 'Xue Wu' THEN 11
WHEN 'Monica Yang' THEN 12
WHEN 'Lily Chen' THEN 13
WHEN 'Christina Liao' THEN 14
WHEN 'Tong Wang' THEN 15
WHEN 'Perry Bian' THEN 16
WHEN 'Sifan Huai' THEN 17
WHEN 'Joy Pu' THEN 18
WHEN 'Sophia Zhuo' THEN 19
WHEN 'Yuki Wu' THEN 20
WHEN 'Daisy Zhu' THEN 21
WHEN 'Dorothy Zhu' THEN 22
WHEN 'Olivia Ma' THEN 23
WHEN 'Haofu Li' THEN 24
WHEN 'Patrick Li' THEN 25
WHEN 'Aaron Guo' THEN 26
WHEN 'Lynn Zheng' THEN 27
WHEN 'Claire Li' THEN 28
WHEN 'Eric Duan' THEN 29
WHEN 'Erin Tang' THEN 30
WHEN 'Shelly Wei' THEN 31
WHEN 'Angie Shen' THEN 32
WHEN 'Viona Li' THEN 33
WHEN 'Ariel Zhang' THEN 34
WHEN 'Shen Jing' THEN 35
WHEN 'Denise Wu' THEN 36
WHEN 'Tina Du' THEN 37
WHEN 'Calvin Hu' THEN 38
WHEN 'Zora Yu' THEN 39
WHEN 'Sharon Shen' THEN 

In [ ]:
mydict={}
for i in range(len(data)):
    m1=str(data['Mentor Name'][i]).strip()
    old1=str(data['old mentor'][i]).strip()
    # m2=data['MenteeName'][i]
    # nm2=data['new mentee'][i]
    # flag=True if(m1=='nan' or nm1=='nan' or str(m2)=='nan' or str(nm2)=='nan') else False
    # if (str(m1)!='nan' and str(nm1)!='nan' and m1 != nm1):
    if(str(old1)!='nan' and m1!=old1):
        pk=int(data['pk'][i])
        mydict.update({i+3:pk})
mydict

{10: 115, 28: 140, 29: 101, 37: 144, 40: 141, 41: 77, 46: 27, 56: 105}

In [ ]:
for k,v in mydict.items():
    print("line:{}, pk:{}".format(k,v))

line:10, pk:115
line:28, pk:140
line:29, pk:101
line:37, pk:144
line:40, pk:141
line:41, pk:77
line:46, pk:27
line:56, pk:105


In [ ]:
m1=str(data['Mentor Name'][8]).strip()
nm1=str(data['new mentor'][8]).strip()
m1 == nm1

True

In [ ]:
filename=open('data/ssby.txt','r',encoding='utf-8')
filename

<_io.TextIOWrapper name='data/ssby.txt' mode='r' encoding='utf-8'>

In [ ]:
filename.read()

In [ ]:
date_ls=[]
sqlf=sql5+" where s1.StaffName_English='{}' and s2.StaffName_English='{}'"
sqlls=[]
# ['Mentor Name', 'MenteeName', 'Start Date', 'End Date', 'Status', 'old date']
for i in range(len(data)):
    status=str(data['Status'][i]).strip()
    date1=str(data['Start Date'][i]).strip()
    date2=str(data['old date'][i]).strip()
    n1=str(data['Mentor Name'][i]).strip()
    n2=str(data['MenteeName'][i]).strip()
    if(date1!=date2):
        date_ls.append(i+3)
    if(status=='未激活'):
        sql=sqlf.format(n1,n2)
        sqlls.append(sql)
print(len(date_ls),len(sqlls))


3 43


In [ ]:
for sql in sqlls:
    print(sql)

In [ ]:
data[:5]

,mentorname,HR_StaffInfoID,menteename,MenteeID
0,Aaron Guo,72.0,Stella Yang,1347.0
1,Aaron Guo,72.0,Arlene Xu,1328.0
2,Alice He,42.0,Christina Liao,1301.0
3,Carol Cai,7.0,Sifan Huai,1322.0
4,Carol Cai,7.0,Joy Pu,1336.0


In [ ]:
for i in range(len(data)):
    # ['mentorname', 'HR_StaffInfoID', 'menteename', 'MenteeID']
    name1=str(data['mentorname'][i]).strip()
    data['HR_StaffInfoID'][i]=pk1=staff_dict[name1]
    name2=str(data['menteename'][i]).strip()
    data['MenteeID'][i]=pk2=staff_dict[name2]
    print(pk1)

In [ ]:
pk_upd_list=[
    115,
    140,
    101,
    144,
    141,
    77,
    27,
    105
]
pk_upd_list

[115, 140, 101, 144, 141, 77, 27, 105]

In [ ]:
sql_upd="update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID={}"
for pk in pk_upd_list:
    print(sql_upd.format(pk))

update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=115
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=140
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=101
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=144
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=141
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=77
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=27
update t_HR_Mentee set CurrentStatus=0 where HR_MenteeID=105


In [ ]:
# ['mentorname', 'HR_StaffInfoID', 'menteename', 'MenteeID', 'old_mentor_name', 'old_mentee_name']
for i in range(8):
    name1=str(data['old_mentor_name'][i]).strip()
    pk1=staff_dict[name1]
    name2=str(data['old_mentee_name'][i]).strip()
    pk2=staff_dict[name2]
    print(pk2)

155
72
149
1244
1247
56
111
90


In [ ]:
sqldata_file='data/pydata_sql.xlsx'
sqldata=pd.read_excel(io=sqldata_file,header=0)
print(sqldata.columns.values)
oldid_ls=[]
newid_ls=[]
# len(sqldata['excel_ID'])

['excel_ID' 'sql_ID' 'mentor ID' 'Mentor Name' 'Mentor C_Name' 'Mentee ID'
 'Mentee Name' 'Mentee C_Name' 'StartDate' 'EndDate' 'CurrentStatus']


In [ ]:
# sqldata['excel_ID'][79]
# sqldata['excel_ID'][80]
for i in range(80):
    oldid=int(sqldata['excel_ID'][i])
    newid=sqldata['sql_ID'][i]
    oldid_ls.append(oldid)
    newid_ls.append(newid)
for i in (79,80):
    oldid_ls.append(int(sqldata['excel_ID'][i]))
print(len(oldid_ls),len(set(oldid_ls)))
print(len(newid_ls),len(set(newid_ls)))

164 81
160 81


In [ ]:
set(oldid_ls)-set(newid_ls)

{74, 96}

In [ ]:
# ['mentorname', 'HR_StaffInfoID', 'menteename', 'MenteeID', 'StartDate', 'IsDeleted', 'CreateDate', 'CreateUserID', 'CurrentStatus']
sql_ins1="INSERT INTO table_name ('HR_StaffInfoID','MenteeID','StartDate','IsDeleted','CreateDate','CreateUserID', 'CurrentStatus')VALUES ({0},{1},{2},{3},{4},{5},{6});"
for i in range(len(data)):
    d1=data['HR_StaffInfoID'][i]

In [ ]:
myfile='data/pydata.xlsx'
data=pd.read_excel(io=myfile,header=0)
name_list=[]
for i in range(len(data)):
    # array(['StaffName_Chinese', 'StaffName_English', 'E_Name', 'AD_Account', 'Code'], dtype=object)
    name=str(data['StaffName_English'][i])
    # name=str(data['E_Name'][i])
    # nl=name.split()
    # name_list.append(nl)
    name_a = str(data['AD_Account'][i])
    name_b=str(data['Code'][i])
    nl=name_a.split('.')
    name_list.append(nl)
    if(name_a!=name_b):
        print(str(i+2)+", "+name_a+", "+name_b)
    

23, Claire.Li, Claire.LI
38, M.LU, M.Lu
82, Dorothy.Zhu, Dorothy.zhu
102, Alice.He, Alice.He001
105, Sherry.Yuan, sherry.yuan
111, Jiayi.Zhou, JiaYi.Zhou
116, Claire.Zhang, Claire.Zhang001
118, Simin.Qu, Qu
121, July.Lin, July.Lin001
135, Jenny.Zhang, Jenny.Zhang001
136, Adele.Wu, Adele.Wu001
137, Evan.Zhou, Evan.Zhou001
138, Carol.Cai, Carol.Cai001
139, Faye.Tang, Faye.Tang001
140, Matthew.Lu, Matthew.Lu001
141, Matthew.Lu, Lu Yun
142, SETHI.ASHOK, nan


In [ ]:
for i in range(len(name_list)):
    nl=name_list[i]
    # print(':'.join(nl))
    for item in nl:
        n=ord(item[0])
        n1=ord(item[1]) if len(item)>1 else 1
        n=str(item[1]) if len(item)>1 else '1'
        # if(not(n>=65 and n<=90) or not(n1>=97 and n1<=122)):
        if(not(item[0]>='A' and item[0]<='Z') or not(item[1]>='a' and item[1]<='z')):
        # if(not(n>=65 and n<=90)):
            print(str(i+2)+", "+item)

In [ ]:
ls1=[]
ls2=[]
for i in range(len(data)):
    # array(['StaffName_Chinese', 'StaffName_English', 'E_Name', 'AD_Account', 'Code'], dtype=object)
    n1=str(data['StaffName_English'][i])
    n2=str(data['AD_Account'][i])
    ls1.append(n2)
    ls2.append('.'.join(n1.split()))

In [ ]:
for i in range(len(ls1)):
    print(str(i+2)+", "+ls1[i]+", "+ls2[i])
    
    # if(ls1[i]!=ls2[i]):
    #     print(str(i+2)+", "+ls1[i]+", "+ls2[i])

In [ ]:
file1='data/true.xlsx' # true
file2='data/test.xlsx' # test
data1=pd.read_excel(io=file1,header=0)
data2=pd.read_excel(io=file2,header=0)
print(data1.columns.values)
# print(data2.columns.values)

['HR_StaffInfoID' 'StaffName_English' 'AD_Account' 'HR_StaffInfoID.1'
 'StaffNumber' 'StaffName_Chinese' 'StaffName_English.1' 'AD_Account.1'
 'Sex' 'IDNumber' 'PassPortNumber' 'HKPassport' 'BirthDay' 'HomePhone'
 'PhoneNumber' 'PersonalEmail' 'Nationality' 'Nation' 'Birthplace'
 'PoliticalStatus' 'PhotoUrl' 'MarriageStatus' 'ResidenceAddress'
 'PermanentAddress' 'EmergencyContactPerson' 'EmergencyContactPersonPhone'
 'CompanyID' 'DepartmentID' 'PositionID' 'LineManager' 'ExtensionNumber'
 'CompanyEmail' 'StaffStatus' 'StaffType' 'EntryDate' 'LeaveDate'
 'VacationDays' 'UpdateDate' 'CreateDate' 'IsDeleted' 'UpdateUserId'
 'CreatedUserId' 'IsTeamLeader' 'IsHaveChild' 'MappingERPID'
 'IDNumberExpireDate' 'PassportNumberExpireDate' 'HKPassportExpireDate'
 'Hukou' 'IsAuthorized']


In [ ]:
for i in range(324):
    name1=str(data1['StaffName_English'][i]).strip()
    name2=str(data2['StaffName_English'][i]).strip()
    id1=str(data1['HR_StaffInfoID'][i])
    id2=str(data2['HR_StaffInfoID'][i])
    if(name1!=name2):
        print("true: "+id1+':'+name1+"\ttest "+id2+':'+name2)


true: 201:Gina Fei	test 201:Yulin Fei
true: 1255:Chelsie Chen	test 1255:Xi Chen1
true: 1272:Jane Zhang	test 1272:Han Zhang
true: 1275:Rebecca Zhou	test 1275:Rui Zhou
true: 1277:Olivia Chang	test 1277:Chuting Chang
true: 1291:Tina Du	test 1291:Zeyuan Du
true: 1307:Mieko Feng	test 1307:Jiaye Feng


In [ ]:
mystr="update t_HR_StaffInfo set AD_Account='{ad}' where StaffName_English='{name}'"
for i in range(len(data1)):
    ad=str(data1['AD_Account'][i]).strip()
    name=str(data1['StaffName_English'][i]).strip()
    print(mystr.format(ad=ad,name=name))

In [ ]:
adfile='data/nullad.xlsx'
addata=pd.read_excel(io=adfile,header=0)
addata.columns.values

array(['HR_StaffInfoID', 'StaffName_English', 'AD_Account',
       'HR_StaffInfoID.1', 'StaffNumber', 'StaffName_Chinese',
       'StaffName_English.1', 'AD_Account.1', 'Sex', 'IDNumber',
       'PassPortNumber', 'HKPassport', 'BirthDay', 'HomePhone',
       'PhoneNumber', 'PersonalEmail', 'Nationality', 'Nation',
       'Birthplace', 'PoliticalStatus', 'PhotoUrl', 'MarriageStatus',
       'ResidenceAddress', 'PermanentAddress', 'EmergencyContactPerson',
       'EmergencyContactPersonPhone', 'CompanyID', 'DepartmentID',
       'PositionID', 'LineManager', 'ExtensionNumber', 'CompanyEmail',
       'StaffStatus', 'StaffType', 'EntryDate', 'LeaveDate',
       'VacationDays', 'UpdateDate', 'CreateDate', 'IsDeleted',
       'UpdateUserId', 'CreatedUserId', 'IsTeamLeader', 'IsHaveChild',
       'MappingERPID', 'IDNumberExpireDate', 'PassportNumberExpireDate',
       'HKPassportExpireDate', 'Hukou', 'IsAuthorized',
       'IsRequireUpdate_Entry', 'IsRequireUpdate_Leave'], dtype=object)

In [ ]:
mystr="update t_HR_StaffInfo set AD_Account='{ad}' where StaffName_English='{name}'"
for i in range(len(addata)):
    name=str(addata['StaffName_English'][i]).strip()
    ad=name.replace(' ','.',2)
    print(mystr.format(ad=ad,name=name))

In [ ]:
file1='data/standard.xlsx' # true
file2='data/test.xlsx' # test
data1=pd.read_excel(io=file1,header=0)
data2=pd.read_excel(io=file2,header=0)
print(data1.columns.values)
print(data2.columns.values)

['No.' '直线号码段' '分机号' 'Name' 'Name.1' '备注']
['No.' 'Name-E' 'Name-C' '分机号' '状态']


In [ ]:
for i in range(len(data1)):
    name1=str(data1['Name'][i])
    standard=str(data1['分机号'][i])
    for j in range(len(data2)):
        if(name1==data2['Name-E'][j]):
            test=str(data2['分机号'][j])
            # if(standard!=test):
            #     print(standard+"\t"+test)
            print(standard+"\t"+test)

In [ ]:
taskfile='data/task.xlsx'
taskdata=pd.read_excel(io=taskfile,header=0)
print(taskdata.columns.values)

['No.' 'Name-E' 'Name-C' 'Name-E.1' '打印机端口号' '状态']


In [ ]:
for i in range(len(taskdata)):
    name1=str(taskdata['Name-E'][i])
    name2=str(taskdata['Name-E.1'][i])
    if(name1!=name2):
        print(str(i)+"\t"+name1+"\t"+name2)

39	Iris Xu	nan
40	Issac Ku	Iris Xu
41	Jane Li	Issac Ku
42	Jane Zhang	Jane Li
43	Janice Wang	Jane Zhang
44	Jason Pan	Janice Wang
45	Jenny Zhang	Jason Pan
46	Jerry Jiao	Jenny Zhang
47	Jessie Sun	Jerry Jiao
48	Jessy Liu	Jessie Sun
49	Jiajia Hao	Jessy Liu
50	Jiayi Zhou	Jiajia Hao
51	Joy Pu	Jiayi Zhou
52	July Lin	Joy Pu
53	Kang Su	July Lin
54	Karen Duan	Kang Su
55	Karen Liu	Karen Duan
56	Kaya Zhang	Karen Liu
57	Kelly Chen	Kaya Zhang
58	Kevin Cai	Kelly Chen
59	Lace Zhang	Kevin Cai
60	Leo Li	Lace Zhang
61	Leon Duan	Leo Li
62	Leslie Wang	Leon Duan
63	Lilith Hua	Leslie Wang
64	Lily Chen	Lilith Hua
65	Lily Wang	Lily Chen
66	Lynn Chen	Lily Wang
67	Lynn Zheng	Lynn Chen
68	M LU	Lynn Zheng
69	Matthew Lu	M LU
70	Mavis Tong	Matthew Lu
71	Mia Wei	Mavis Tong
72	Mieko Feng	Mia Wei
73	Mira Ma	Mieko Feng
74	Monica Yang	Mira Ma
75	Naomi Chen	Monica Yang
76	Neal Teng	Naomi Chen
77	Neo Wu	Neal Teng
78	Nerissa Sun	Neo Wu
79	Nora Xu 	Nerissa Sun
80	Olivia Chang	Nora Xu 
81	Olivia Ma	Olivia Chang
82	Oorain Ju	Ol

In [ ]:
i=11

import calendar


cal=calendar.Calendar()
n=0
dates=cal.itermonthdates(2020,i)
for d in dates:
    if(n%7==0):
        print("\n\n\n\n\n")
    mystr=d.strftime('%m月 %d日')
    print(mystr,end="\t\t")
    n=n+1







10月 26日		10月 27日		10月 28日		10月 29日		10月 30日		10月 31日		11月 01日		





11月 02日		11月 03日		11月 04日		11月 05日		11月 06日		11月 07日		11月 08日		





11月 09日		11月 10日		11月 11日		11月 12日		11月 13日		11月 14日		11月 15日		





11月 16日		11月 17日		11月 18日		11月 19日		11月 20日		11月 21日		11月 22日		





11月 23日		11月 24日		11月 25日		11月 26日		11月 27日		11月 28日		11月 29日		





11月 30日		12月 01日		12月 02日		12月 03日		12月 04日		12月 05日		12月 06日		

In [ ]:
from xml.dom.minidom import parse
# from urllib.request import urlopen
# from xml.etree.ElementTree import parse

In [ ]:
dom=parse('data/IPOFFICESD_Users.xml')
data=dom.documentElement
users=data.getElementsByTagName('tns:user')

In [ ]:
from pandas import Series,DataFrame
import pandas as pd
!pip install openpyxl
import openpyxl

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 245kB 4.6MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8917 sha256=8e87ea7f83d6e9ff744c00d87622bd2bff725798bbc025d1bc4aa056ca487896
  Stored in directory: /home/aistudio/.cache/pip/wheels/22/08/69/f546610376ad227cb3f48c8c39dec786990fdbc961dc69ff9d
Successfully built et-xmlfile


In [ ]:
df=pd.DataFrame(columns=["E_Name","C_Name","extension"])

In [ ]:
for user in users:
    loginname=user.getElementsByTagName('loginName')[0].childNodes[0].nodeValue.strip("@user").split(' ')
    extension=user.getElementsByTagName('csm:extension')[0].childNodes[0].nodeValue
    ename=loginname[0]
    cname=""
    if(len(loginname)>2):
        ename=loginname[0]+" "+loginname[1]
        cname=loginname[-1]
    userdict={'E_Name':ename,'C_Name':cname,'extension':extension}
    df=df.append(userdict,ignore_index=True)
    # print(userdict)

datalength=len(users)

In [ ]:
import datetime
now=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename="AVAYA分机号"+str(datalength)+"_"+now+".xlsx"

df.to_excel("work/"+filename)


In [ ]:
import pandas as pd
import os
import numpy as np
! pip install xlrd

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 112kB 10.1MB/s eta 0:00:01


In [ ]:
cardfile='data/card0818.xlsx'
cardinfo=pd.read_excel(io=cardfile,header=0)
extenfile='data/143_20200918_133203.xlsx'
extdata=pd.read_excel(io=extenfile,header=0)

print(len(cardinfo))
print(cardinfo.columns.values)
print(len(extdata))
print(extdata.columns.values)

131
['序号' '部门' '中文名' '英文名' '中文职位' '英文职位' '总机' '分机号' '手机' '邮箱' '名片份数']
143
['E_Name' 'C_Name' 'extension']


In [ ]:
for i in range(len(cardinfo)):
    card_ename=str(cardinfo['英文名'][i]).strip()
    card_ext=int(cardinfo['分机号'][i])
    for j in range(len(extdata)):
        ext_ename=str(extdata['E_Name'][j]).strip()
        ext_extnum=int(extdata['extension'][j])
        # if((card_ename==ext_ename) and str(card_ext)!=str(ext_extnum)):
            # print(card_ename+'\tcard: '+str(card_ext)+'\tsystem: '+str(ext_extnum))
        if((card_ext==ext_extnum) and (card_ename!=ext_ename)):
            print(str(card_ext)+',\t'+str(card_ename)+',\t'+ext_ename)

8116,	Frances Zhang,	Frances Z
8077,	SETHIASHOK,	SETHI ASHOK
8152,	Serena Zhang,	Serena Z
8141,	Gina Fei,	Yulin Fei
8122,	Francis Huang,	Francis H
8136,	Olivia Chang,	Chuting C
8135,	Steven Shuai,	Steven
8121,	Gabrielle Guan,	Gabrielle G
8137,	Christina Liao,	Christina L
8049,	Claire Zhang,	Claire Z
8133,	Sophie Chai,	SophieChai


In [ ]:
set_cardext=set(cardinfo['分机号'])
set_syext=set(extdata['extension'])
subSet=set_syext-set_cardext
print(subSet)
for j in range(len(extdata)):
    ext_ename=str(extdata['E_Name'][j]).strip()
    ext_extnum=int(extdata['extension'][j])
    if(ext_extnum in subSet):
        print(ext_ename+'\t'+str(ext_extnum))

{8000, 8103, 8072, 8105, 8074, 8108, 8109, 8111, 8112, 8113, 8115, 8053, 8091, 8092}
Newton	8053
Einstein	8072
Galileo	8074
Franklin	8091
Da	8092
Hugo	8103
Bacon	8105
Mozart	8108
Goeth	8109
Haydn	8112
Copernic	8111
Switchboard	8000
Jane	8113
Joe Zhu	8115


In [ ]:
myfile='data/分机号_20200918_170104.xlsx'
mydata=pd.read_excel(io=myfile,header=0)

print(len(mydata))
print(mydata.columns.values)

159
['序号' '连续' 'loginName' 'C_Name' 'extension' '分机号' '中文名' '英文名']


In [ ]:
for i in range(len(mydata)):
    sy_name=str(mydata['C_Name'][i]).strip()
    card_name=str(mydata['中文名'][i]).strip()
    sy_ext=str(mydata['extension'][i]).strip()
    card_ext=str(mydata['分机号'][i]).strip()
    if((sy_name!=card_name) or (sy_ext!=card_ext)):
        output="{0},{1},{2},{3}".format(sy_name,card_name,sy_ext,card_ext)
        print(output)

nan,nan,8000.0,nan
金振末,金振未,8026.0,8026.0
nan,王卉,nan,8031.0
卞鹏瑞,卞鹏睿,8036.0,8036.0
nan,杜泽元,nan,8038.0
nan,nan,8053.0,nan
许沺,徐沺,8065.0,8065.0
封佳钰,封加烨,8067.0,8067.0
nan,nan,8072.0,nan
nan,nan,8074.0,nan
nan,nan,8091.0,nan
nan,nan,8092.0,nan
nan,nan,8103.0,nan
nan,nan,8105.0,nan
nan,nan,8108.0,nan
nan,nan,8109.0,nan
nan,nan,8111.0,nan
nan,nan,8112.0,nan
张吴江,nan,8113.0,nan
滕赟,滕贇,8114.0,8114.0
朱萍,nan,8115.0,nan


In [ ]:
from pandas import Series,DataFrame
import pandas as pd
!pip install openpyxl
import openpyxl
!pip install xlrd

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [ ]:
df=pd.DataFrame(columns=["连续","分机号","英文名","中文名","loginName","备注"])

In [ ]:
cardfile='data/card0818.xlsx'
cardinfo=pd.read_excel(io=cardfile,header=0)
extenfile='data/143_20200919_004133.xlsx'
extdata=pd.read_excel(io=extenfile,header=0)

print(len(cardinfo))
print(cardinfo.columns.values)
print(len(extdata))
print(extdata.columns.values)

131
['序号' '部门' '中文名' '英文名' '中文职位' '英文职位' '总机' '分机号' '手机' '邮箱' '名片份数']
143
['loginName' 'E_Name' 'C_Name' 'extension']


In [ ]:
remark=ename=cname=loginName=extension=""
    # 1. 根据ext匹配data里的数据
    # 2. 填入ename cname loginname 
    # 3. loginname: 'Switchboard'->前台
    #               ['Hugo','Bacon','Mozart','Goeth','Copernic','Haydn']->电话间
    #               ['Newton','Einstein','Galileo','Franklin','Da Vinci']->会议室
ls_r1=['Newton','Einstein','Galileo','Franklin','Da Vinci']
ls_r2=['Hugo','Bacon','Mozart','Goeth','Copernic','Haydn']

In [ ]:
for index in range(200):
    indexEXT=8000+index
    for i in range(len(extdata)):
        sy_ext=int(extdata['extension'][i])
        if(sy_ext==indexEXT):
            loginName=str(extdata['loginName'][i]).strip()
            if loginName in ls_r1:
                remark="会议室"
            elif loginName in ls_r2:
                remark="电话间"
            else:
                remark="前台" if(loginName=="Switchboard") else ''
            break
    for j in range(len(cardinfo)):
        card_ext=int(cardinfo['分机号'][j])
        if(card_ext==indexEXT):
            ename=str(cardinfo['英文名'][j]).strip()
            cname=str(cardinfo['中文名'][j]).strip()
            break
    if(sy_ext==card_ext):
        extension=sy_ext
    if(loginName==''):
        remark="留空"
    userdict={'连续':indexEXT,'分机号':extension,'英文名':ename,'中文名':cname,'loginName':loginName,'备注':remark}
    df=df.append(userdict,ignore_index=True)
    remark=ename=cname=loginName=extension=""


In [ ]:
import datetime
now=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename="分机号_"+now+".xlsx"

df.to_excel("work/"+filename)

In [ ]:
from sympy import *

a3=Symbol('a3')
a4=Symbol('a4')
a5=Symbol('a5')

val=solve([a3+a4+a5-1,a3*50+a4*250+a5*2000-256],[a3,a4,a5])
val

{a4: 103/100 - 39*a5/4, a3: 35*a5/4 - 3/100}

In [ ]:
import pandas as pd

ls=[]
a5=0.001
while(a5<0.01):
    a4=103/100 - 39*a5/4
    a3=35*a5/4 - 3/100
    ls.append([a3,a4,a5])
    a5+=0.001


In [ ]:
ls

In [ ]:
dom=parse('data/convertSwf_0_5079.xml')
data=dom.documentElement
sn_dom=data.getElementsByTagName('sn')
status_dom=data.getElementsByTagName('status')
fileID_dom=data.getElementsByTagName('fileID')
title_dom=data.getElementsByTagName('title')
fileName_dom=data.getElementsByTagName('fileName')
fileType_dom=data.getElementsByTagName('fileType')
createTime_dom=data.getElementsByTagName('createTime')


In [ ]:
import pandas as pd
df=pd.DataFrame(columns=["sn","status","fileID","title","fileName","fileType","createTime"])
for i in range(len(sn_dom)):
    v1=sn_dom[i].childNodes[0].nodeValue
    v2=status_dom[i].childNodes[0].nodeValue
    v3=title_dom[i].childNodes[0].nodeValue
    v4=fileID_dom[i].childNodes[0].nodeValue
    v5=v6=""
    if(i not in (3621,3633,3699,3718,3761,3837,3997,4081,4102,4165)):
        v5=fileType_dom[i].childNodes[0].nodeValue
    if(i!=204):
        v6=fileName_dom[i].childNodes[0].nodeValue
    v7=createTime_dom[i].childNodes[0].nodeValue
    mydict={'sn':v1,'status':v2,'fileID':v4,'title':v3,'fileName':v6,'fileType':v5,'createTime':v7}
    df=df.append(mydict,ignore_index=True)

In [ ]:
df.to_excel("export.xlsx")

In [ ]:
df.head()

,sn,status,fileID,title,fileName,fileType,createTime
0,0,Success,30168,新型冠状病毒影响之下的营销趋势预判与洞察-蓝标-20200220.pdf,新型冠状病毒影响之下的营销趋势预判与洞察-蓝标-20200220,pdf,2020/10/16 23:53:38
1,1,Success,30169,见实科技-公域私域新组合.pdf,见实科技-公域私域新组合,pdf,2020/10/16 23:54:33
2,2,Success,30170,阿里VS京东VS拼多多：三大平台之对比分析-分级、竞争、进化-2020.5.pdf,阿里VS京东VS拼多多：三大平台之对比分析-分级、竞争、进化-20205,5,2020/10/16 23:54:52
3,3,Success,30171,1_苏世民我的经验与教训.pdf,1_苏世民我的经验与教训,pdf,2020/10/16 23:56:53
4,4,Success,30172,BCG-假说驱动管理.pdf,BCG-假说驱动管理,pdf,2020/10/16 23:58:05


In [ ]:
cs=cf=0
for c in df.status:
    cs+=(1 if c=='Success' else 0)
    cf+=(1 if c=='Fail' else 0)

In [ ]:
print(cs,cf)

817 4263


In [ ]:
dom=parse('data/FeedBack_1104_2238.xml')
data=dom.documentElement
fbs=data.getElementsByTagName('FeedBack')

In [ ]:
import pandas as pd
df=pd.DataFrame(columns=["sn","status","fileID","title","fileName","fileType","createTime"]) 
ls_file_not_exist=[]
ls_copyFail=[]

In [ ]:
for fb in fbs:
    sn=fb.getElementsByTagName('sn')[0].childNodes[0].nodeValue
    status=fb.getElementsByTagName('status')[0].childNodes[0].nodeValue
    fileID=fb.getElementsByTagName('fileID')[0].childNodes[0].nodeValue
    title=fb.getElementsByTagName('title')[0].childNodes[0].nodeValue
    fileName=fb.getElementsByTagName('fileName')[0].childNodes[0].nodeValue
    # print(len(fb.getElementsByTagName('fileType')[0].childNodes))
    if(len(fb.getElementsByTagName('fileType')[0].childNodes)>0):
        fileType=fb.getElementsByTagName('fileType')[0].childNodes[0].nodeValue
    else:
        fileType=title.split('.')[-1]
    createTime=fb.getElementsByTagName('createTime')[0].childNodes[0].nodeValue
    if("exist" in status):
        ls_file_not_exist.append(fileID)
    if("Fail" in status and fileType in ["doc","docx","xls","xlsx","ppt","pptx","pdf"]):
        ls_copyFail.append(fileID)
    fbdict={"sn":sn,"status":status,"fileID":fileID,"title":title,"fileName":fileName,"fileType":fileType,"createTime":createTime}
    df=df.append(fbdict,ignore_index=True)
    # print(userdict)
    # print(sn)

datalength=len(fbs)

In [ ]:
df.to_excel("feedBack_pms.xlsx")

In [ ]:
# work/output.txt
f_op = open("work/output.txt", 'w+',encoding="utf-8")
print(ls_copyFail,file=f_op)
f_op.close()

In [ ]:
len(ls_copyFail)

61